In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

In [2]:
data,data_info = tfds.load(name='plant_village',as_supervised=True,split='train',with_info=True,)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/plant_village/1.0.2.incomplete6ZII66/plant_village-train.tfrecord


  0%|          | 0/54303 [00:00<?, ? examples/s]

Dataset plant_village downloaded and prepared to /root/tensorflow_datasets/plant_village/1.0.2. Subsequent calls will reuse this data.


In [3]:
n_val=0.1*data_info.splits['train'].num_examples
n_val=tf.cast(n_val,tf.int64)

In [4]:
def scale(image,label):
  im=tf.cast(image,tf.float32)
  im/=255.0
  return im,label

scaled_data=data.map(scale)

In [5]:
val_data=scaled_data.take(n_val).batch(32).prefetch(1)
train_data=scaled_data.skip(n_val).batch(32).prefetch(1)

In [7]:
from keras.layers import Flatten,Dense

base_model =tf.keras.applications.resnet50.ResNet50(include_top=False,input_shape=(256,256,3))

x= Flatten()(base_model.layers[-1].output)
x= Dense(1024,activation='relu')(x)
x= Dense(512,activation='relu')(x)
x= Dense(128,activation='relu')(x)
x= Dense(38, activation= 'softmax')(x)

94781440/94765736 [==============================] - 1s 0us/step


In [8]:
model = tf.keras.models.Model(inputs= base_model.input, outputs = x)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(train_data,epochs=10,validation_data=val_data)

Epoch 1/10
1528/1528 [==============================] - 1464s 943ms/step - loss: 0.8238 - accuracy: 0.7858 - val_loss: 1.0458 - val_accuracy: 0.7366
Epoch 2/10
1528/1528 [==============================] - 1443s 942ms/step - loss: 0.2380 - accuracy: 0.9304 - val_loss: 0.7700 - val_accuracy: 0.7974
Epoch 3/10
 942/1528 [=================>............] - ETA: 8:53 - loss: 0.1798 - accuracy: 0.9480